In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pylab
import csv
import warnings
warnings.filterwarnings('ignore')

# Read in the data

In [2]:
match = pd.read_csv('match_data.csv').fillna(0).iloc[:, 1:]
player_ability = pd.read_csv('ranked_player_data.csv').iloc[:, 1:]
players = pd.read_csv('player_data.csv').fillna(0).iloc[:, 1:]
#events = pd.read_csv('event_data.csv').fillna(0).iloc[:, 1:]
players.rename(columns={'id': 'match_id'}, inplace=True) #conform the Id column across dataframes
players = players.iloc[:, [5, 0, 6, 15, 16, 28, 17, 29, 30, 1, 2, 3, 4, 7, 8, 9, 10, 11, 12, 13, 14, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 31, 32, 33, 34, 35, 36, 37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58]]
#events = events.iloc[:, [9, 0, 1, 2, 3, 4, 5, 6, 7, 8]] #reorder match_id to front

#drop cancelled events (loses about 81)
match.drop(match[match['match_status']=='Cancelled'].index, inplace=True)

a = match.iloc[:, 0:26]
b = match.iloc[:, [27, 29, 30, 31, 32, 33, 34, 35, 36, 38, 42, 44, 45, 46, 52, 54, 58, 67, 69, 70, 71, 72, 73, 74, 75, 76, 78, 82, 84, 85, 86, 92, 94, 98]]
match_2 = pd.concat([a,b], axis=1)

#remove point scoring kicks. Calculate conversion based off conv. rate * tries scored. Penalty rate * penalties conceded
match_3 = match_2.drop(match_2.iloc[:, [36, 37, 38, 29, 53, 54, 55, 56]], axis =1)

# Merge player ability with players and calculate each match team ability score.

In [8]:
#manually select the team player into a list. Select players in their position order in the list.
player_df = players[players['season'] > 2021]
player_df2 = player_df[['player_id', 'position', 'name']]

player_df['outcome'] = np.where(player_df['home_score'] > player_df['away_score'], 1, 0)
#players['team'] = np.where(players['team'] == 'home', 1, 0)
players_test = player_df[['match_id', 'player_id', 'team', 'outcome']]
merged_df = pd.merge(players_test, player_ability, on = 'player_id', how = 'left')
#merged_df.sort_values(by ='match_id', inplace=True)
merged_df.reset_index(inplace=True)

#1 - home wins, 0 = away won

#null where ranks == 0; the players sometimes occur more than once, so we want to keep the rows, but make them have no impact
merged_df['attacking_rank'] = np.where(merged_df['Ability_score'] == 0, 0, merged_df['attacking_rank'])
merged_df['defending_rank'] = np.where(merged_df['Ability_score'] == 0, 0, merged_df['defending_rank'])

merged_home = merged_df.loc[(merged_df['team'] == 'home')]
merged_home['team'] = 1
merged_home.rename(columns = {'Ability_score':'home_ability_score', 'attack_score':'home_attack_score', 'defend_score':'home_defend_score', 'attacking_rank':'home_attacking_rank', 'defending_rank':'home_defending_rank'}, inplace=True)

merged_away = merged_df.loc[(merged_df['team'] == 'away')]
merged_away['team'] = 0
merged_away.rename(columns = {'Ability_score':'away_ability_score', 'attack_score':'away_attack_score', 'defend_score':'away_defend_score', 'attacking_rank':'away_attacking_rank', 'defending_rank':'away_defending_rank'}, inplace=True)

#now need to sum by match ID
merged_h = merged_home.drop(columns = ['index', 'player_id', 'name'])
merged_hgroup = merged_h.groupby('match_id').sum()
merged_hgroup['team'] = 1
merged_hgroup['outcome'] = np.where(merged_hgroup['outcome'] > 0, 1, 0)
merged_hgroup.reset_index(inplace=True)

merged_a = merged_away.drop(columns = ['index', 'player_id', 'name'])
merged_agroup = merged_a.groupby('match_id').sum()
merged_agroup['team'] = 0
merged_agroup['outcome'] = np.where(merged_hgroup['outcome'] > 0, 1, 0)
merged_agroup.reset_index(inplace=True)


#drop one outcome column, 0 means away team won, 1 means home team won, drop team column. Merge on match ID
#then can use to predict. 

match_df = pd.merge(merged_hgroup,merged_agroup, on = ['match_id', 'outcome'], how = 'outer')
match_df.drop(columns = ['team_x', 'team_y'], inplace=True)

match_df.head(100)

,match_id,outcome,home_ability_score,home_attack_score,home_defend_score,home_attacking_rank,home_defending_rank,away_ability_score,away_attack_score,away_defend_score,away_attacking_rank,away_defending_rank
0,4707106,0,129.90,126.5,133.3,10374.0,10714.0,341.80,350.5,333.1,7313.0,7461.0
1,4707115,1,485.55,445.5,525.6,4325.0,3809.0,315.45,319.1,311.8,6448.0,6773.0
2,4707124,1,509.30,519.9,498.7,4184.0,4430.0,317.15,310.8,323.5,6523.0,6449.0
3,4707133,1,487.60,453.5,521.7,4677.0,4660.0,415.85,402.5,429.2,5129.0,5240.0
4,4707142,1,610.20,602.1,618.3,4501.0,4437.0,271.55,255.0,288.1,5731.0,5631.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,8404909,0,535.85,536.3,535.4,4935.0,4898.0,561.35,562.0,560.7,3197.0,3228.0
96,8404918,1,360.25,337.2,383.3,4992.0,5106.0,357.65,338.9,376.4,4796.0,4777.0
97,8404927,1,509.70,478.7,540.7,4377.0,4350.0,324.85,317.2,332.5,6867.0,6905.0
98,8404936,0,339.90,323.7,356.1,5066.0,5099.0,544.75,547.3,542.2,3464.0,3526.0


In [6]:
match_df.to_csv("match_df.csv", header = True)